# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,-11.15,73,75,6.17,CA,1712494767
1,1,puerto natales,-51.7236,-72.4875,6.25,61,40,7.72,CL,1712494772
2,2,tura,25.5198,90.2201,29.25,23,100,0.66,IN,1712494778
3,3,carnarvon,-24.8667,113.6333,24.45,74,0,9.51,AU,1712494641
4,4,isafjordur,66.0755,-23.1240,-1.98,74,75,10.80,IS,1712494784


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
#%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=30)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=30)\
                            &(city_data_df['Country']=='US')\
                            &(city_data_df['Humidity']<70),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

15


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,bethel,41.3712,-73.4140,7.20,67,21,1.79,US,1712494978
141,141,greenville,35.6127,-77.3663,11.19,57,0,3.60,US,1712495209
197,197,marquette,46.5333,-87.6335,2.10,68,5,1.34,US,1712495666
243,243,fort irwin,35.2627,-116.6848,3.74,56,0,3.09,US,1712496028
265,265,redan,33.7454,-84.1316,10.77,55,5,3.62,US,1712496209
272,272,toccoa,34.5773,-83.3324,11.08,55,0,3.09,US,1712496262
362,362,fort bragg,35.1390,-79.0060,12.30,57,0,2.68,US,1712496720
392,392,ontario,34.0633,-117.6509,10.45,69,20,2.06,US,1712497050
405,405,herkimer,43.0256,-74.9860,8.13,65,6,2.24,US,1712497256
443,443,dalhart,36.0595,-102.5133,4.71,35,0,6.69,US,1712497503


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

C:\Users\rsilv\AppData\Local\Temp\ipykernel_6520\3242012003.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
51,bethel,US,41.3712,-73.4140,67,
141,greenville,US,35.6127,-77.3663,57,
197,marquette,US,46.5333,-87.6335,68,
243,fort irwin,US,35.2627,-116.6848,56,
265,redan,US,33.7454,-84.1316,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


bethel - nearest hotel: Hampton Inn Danbury
greenville - nearest hotel: Quality Inn
marquette - nearest hotel: No hotel found
fort irwin - nearest hotel: Landmark Inn
redan - nearest hotel: Fairfield Inn & Suites Atlanta Stonecrest
toccoa - nearest hotel: Simmons-Bond Inn Bed & Breakfast
fort bragg - nearest hotel: Airborne Inn Lodging
ontario - nearest hotel: Travelodge by Wyndham Ontario
herkimer - nearest hotel: Red Roof Inn
dalhart - nearest hotel: La Quinta Inn & Suites by Wyndham Dalhart
rolla - nearest hotel: Pear Tree Inn Rolla
meadow lake - nearest hotel: No hotel found
hazard - nearest hotel: Hampton Inn & Suites
cleveland - nearest hotel: Renaissance Cleveland Hotel
westport - nearest hotel: Norwalk Inn and Conference Center


,City,Country,Lat,Lng,Humidity,Hotel Name
51,bethel,US,41.3712,-73.4140,67,Hampton Inn Danbury
141,greenville,US,35.6127,-77.3663,57,Quality Inn
197,marquette,US,46.5333,-87.6335,68,No hotel found
243,fort irwin,US,35.2627,-116.6848,56,Landmark Inn
265,redan,US,33.7454,-84.1316,55,Fairfield Inn & Suites Atlanta Stonecrest
272,toccoa,US,34.5773,-83.3324,55,Simmons-Bond Inn Bed & Breakfast
362,fort bragg,US,35.1390,-79.0060,57,Airborne Inn Lodging
392,ontario,US,34.0633,-117.6509,69,Travelodge by Wyndham Ontario
405,herkimer,US,43.0256,-74.9860,65,Red Roof Inn
443,dalhart,US,36.0595,-102.5133,35,La Quinta Inn & Suites by Wyndham Dalhart


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
#%%capture --no-display

# Configure the map plot
plot_4 = hotel_df.hvplot.points(
            'Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
plot_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)